In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")
SIZE = 299

In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [7]:
train_dataset_info

array([{'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [8]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
        zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_yellow_ch),
#             zero_data, zero_data), -1)
#         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2))
        image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug


In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

Using TensorFlow backend.


In [10]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [11]:
import tensorflow as tf
from tensorflow.python.ops import array_ops

# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py
def focal_loss_org(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)


In [12]:
def focal_loss(weights=None, alpha=0.25, gamma=2):
    def focal_loss_my(target_tensor, prediction_tensor, ):
        r"""Compute focal loss for predictions.
            Multi-labels Focal loss formula:
                FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                     ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
        Args:
         prediction_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing the predicted logits for each class
         target_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing one-hot encoded classification targets
         weights: A float tensor of shape [batch_size, num_anchors]
         alpha: A scalar tensor for focal loss alpha hyper-parameter
         gamma: A scalar tensor for focal loss gamma hyper-parameter
        Returns:
            loss: A (scalar) tensor representing the value of the loss function
        """
        sigmoid_p = tf.nn.sigmoid(prediction_tensor)
        zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)

        # For poitive prediction, only need consider front part loss, back part is 0;
        # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
        pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)

        # For negative prediction, only need consider back part loss, front part is 0;
        # target_tensor > zeros <=> z=1, so negative coefficient = 0.
        neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
        per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                              - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
        return tf.reduce_sum(per_entry_cross_ent)
#         return K.mean(K.binary_crossentropy(target_tensor, prediction_tensor), axis=-1) + tf.reduce_sum(per_entry_cross_ent)
    return focal_loss_my

In [13]:
def focal_loss_fixed(y_true, y_pred):
    gamma = 2.
    alpha = 0.25
    print(y_pred)
    print(y_true)
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#     pt_1 = K.clip(pt_1, 1e-3, .999)
#     pt_0 = K.clip(pt_0, 1e-3, .999)

    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


In [14]:
# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#         pt_1 = K.clip(pt_1, 1e-3, .999)
#         pt_0 = K.clip(pt_0, 1e-3, .999)

#         return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed

In [15]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../cache/InceptionV3.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [16]:
train_dataset_info.shape

(31072,)

In [17]:

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
# np.random.shuffle(indexes)
# train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
submit = pd.read_csv('../data/sample_submission.csv')

# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
sub_class_preds = np.zeros((submit.shape[0], 28))

fold_ = 0
for train_indexes, valid_indexes in kf.split(indexes):
    _preds = []
    # create train and valid datagens
    train_generator = data_generator.create_train(
        train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
    validation_generator = data_generator.create_train(
        train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

    # warm up model
    model = create_model(
        input_shape=(SIZE,SIZE,3), 
        n_out=28)

    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    model.layers[-3].trainable = True
    model.layers[-4].trainable = True
    model.layers[-5].trainable = True
    model.layers[-6].trainable = True
    
    model.compile(
        loss=f1_loss, 
        optimizer=Adam(1e-03),
        metrics=[f1])
#     model.summary()

    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=2, 
        verbose=1)
    
    # train all layers
    epochs=10
    for layer in model.layers:
        layer.trainable = True
    model.compile(loss=f1_loss,
                optimizer=Adam(lr=1e-4),
                metrics=[f1])
    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=epochs, 
        verbose=1,
        callbacks=callbacks_list)
    
    for idx in tqdm(valid_indexes):
        item = train_dataset_info[idx]
        path = item['path']
        labels = item['labels']
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        oof_class_preds[idx] = score_predict
    
    for idx, name in tqdm(enumerate(submit['Id'])):
        path = os.path.join('../data/test/', name)
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        sub_class_preds[idx] += score_predict
    fold_ += 1
sub_class_preds /= n_splits

Epoch 1/2
1554/1554 [==============================] - 372s 240ms/step - loss: 1.1202 - f1: 0.0373 - val_loss: 1.2955 - val_f1: 0.0395
Epoch 2/2
1554/1554 [==============================] - 360s 231ms/step - loss: 1.1048 - f1: 0.0495 - val_loss: 1.1427 - val_f1: 0.0314
Epoch 1/10
1554/1554 [==============================] - 426s 274ms/step - loss: 1.0607 - f1: 0.0917 - val_loss: 1.0183 - val_f1: 0.1393

Epoch 00001: val_loss improved from inf to 1.01828, saving model to ../cache/InceptionV3.h5
Epoch 2/10
1554/1554 [==============================] - 407s 262ms/step - loss: 0.9904 - f1: 0.1597 - val_loss: 0.9650 - val_f1: 0.2090

Epoch 00002: val_loss improved from 1.01828 to 0.96498, saving model to ../cache/InceptionV3.h5
Epoch 3/10
1554/1554 [==============================] - 409s 263ms/step - loss: 0.9408 - f1: 0.2034 - val_loss: 0.8904 - val_f1: 0.2623

Epoch 00003: val_loss improved from 0.96498 to 0.89041, saving model to ../cache/InceptionV3.h5
Epoch 4/10
1554/1554 [=============

100%|██████████| 6215/6215 [03:45<00:00, 27.58it/s]
11702it [08:43, 22.36it/s]


Epoch 1/2
1554/1554 [==============================] - 370s 238ms/step - loss: 1.1204 - f1: 0.0381 - val_loss: 1.1608 - val_f1: 0.0211
Epoch 2/2
1554/1554 [==============================] - 357s 230ms/step - loss: 1.1044 - f1: 0.0496 - val_loss: 1.1687 - val_f1: 0.0368
Epoch 1/10
1554/1554 [==============================] - 425s 273ms/step - loss: 1.0517 - f1: 0.1024 - val_loss: 0.9814 - val_f1: 0.1723

Epoch 00001: val_loss did not improve from 0.80882
Epoch 2/10
1554/1554 [==============================] - 404s 260ms/step - loss: 0.9802 - f1: 0.1700 - val_loss: 0.9262 - val_f1: 0.2357

Epoch 00002: val_loss did not improve from 0.80882
Epoch 3/10
1554/1554 [==============================] - 402s 259ms/step - loss: 0.9357 - f1: 0.2081 - val_loss: 0.8787 - val_f1: 0.2769

Epoch 00003: val_loss did not improve from 0.80882
Epoch 4/10
1554/1554 [==============================] - 413s 266ms/step - loss: 0.9092 - f1: 0.2280 - val_loss: 0.8452 - val_f1: 0.3064

Epoch 00004: val_loss did not

100%|██████████| 6215/6215 [03:52<00:00, 26.70it/s]
11702it [07:05, 27.48it/s]


Epoch 1/2
1554/1554 [==============================] - 364s 235ms/step - loss: 1.1190 - f1: 0.0404 - val_loss: 1.2429 - val_f1: 0.0265
Epoch 2/2
1554/1554 [==============================] - 352s 227ms/step - loss: 1.1045 - f1: 0.0507 - val_loss: 1.2384 - val_f1: 0.0321
Epoch 1/10
1554/1554 [==============================] - 422s 271ms/step - loss: 1.0544 - f1: 0.0989 - val_loss: 1.0067 - val_f1: 0.1720

Epoch 00001: val_loss did not improve from 0.80470
Epoch 2/10
1554/1554 [==============================] - 396s 255ms/step - loss: 0.9806 - f1: 0.1680 - val_loss: 0.9036 - val_f1: 0.2442

Epoch 00002: val_loss did not improve from 0.80470
Epoch 3/10
1554/1554 [==============================] - 395s 254ms/step - loss: 0.9356 - f1: 0.2062 - val_loss: 0.8960 - val_f1: 0.2617

Epoch 00003: val_loss did not improve from 0.80470
Epoch 4/10
1554/1554 [==============================] - 395s 254ms/step - loss: 0.9116 - f1: 0.2258 - val_loss: 0.8560 - val_f1: 0.2853

Epoch 00004: val_loss did not

...

Epoch 1/2
1554/1554 [==============================] - 438s 282ms/step - loss: 1.1210 - f1: 0.0371 - val_loss: 1.1428 - val_f1: 0.0219
Epoch 2/2
1554/1554 [==============================] - 353s 227ms/step - loss: 1.1069 - f1: 0.0473 - val_loss: 1.1781 - val_f1: 0.0314
Epoch 1/10
1554/1554 [==============================] - 434s 279ms/step - loss: 1.0542 - f1: 0.1000 - val_loss: 0.9851 - val_f1: 0.1721

Epoch 00001: val_loss did not improve from 0.79359
Epoch 2/10
1554/1554 [==============================] - 405s 261ms/step - loss: 0.9828 - f1: 0.1653 - val_loss: 0.9030 - val_f1: 0.2448

Epoch 00002: val_loss did not improve from 0.79359
Epoch 3/10
1554/1554 [==============================] - 408s 263ms/step - loss: 0.9378 - f1: 0.2046 - val_loss: 0.9002 - val_f1: 0.2667

Epoch 00003: val_loss did not improve from 0.79359
Epoch 4/10
1554/1554 [==============================] - 444s 286ms/step - loss: 0.9079 - f1: 0.2282 - val_loss: 0.8606 - val_f1: 0.2883

Epoch 00004: val_loss did not

  0%|          | 0/6214 [00:00<?, ?it/s]


Epoch 00010: val_loss did not improve from 0.79359


100%|██████████| 6214/6214 [04:23<00:00, 23.61it/s]
11702it [08:01, 24.31it/s]


KeyboardInterrupt: 

In [26]:
# oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
# sub_class_preds = np.zeros((submit.shape[0], 28))
# score_predict = model.predict(image[np.newaxis])[0]
# oof_class_preds[idx] = score_predict

In [28]:
idx

6

In [29]:
oof_class_preds[6]

array([0.45820379, 0.12732422, 0.3277137 , 0.23298733, 0.3001917 ,
       0.24295843, 0.02109067, 0.19385408, 0.00467812, 0.00399152,
       0.00271517, 0.06066669, 0.02236799, 0.00866987, 0.03342314,
       0.00158739, 0.03309921, 0.00860063, 0.03825995, 0.12749843,
       0.01946285, 0.02194879, 0.03339204, 0.14109442, 0.02711315,
       0.1113272 , 0.03284218, 0.00121808])

In [18]:
item = train_dataset_info[0]

In [21]:
item['labels']

array([16,  0])

In [ ]:
# for train_index, test_index in kf.split(indexes):
# ...    print("TRAIN:", train_index, "TEST:", test_index)

In [19]:
sub_class_preds.shape

(11702, 28)

In [20]:
predicted = []
for line in tqdm(sub_class_preds):
    label_predict = np.arange(28)[line>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

100%|██████████| 11702/11702 [00:00<00:00, 82674.16it/s]


In [21]:
len(predicted)

11702

In [22]:
predicted

['0 2',
 '0 1 3 4 5 6 11 25',
 '0 2 5 21 25',
 '0 25',
 '0 7 21 23 25',
 '4 12 21 25',
 '0 4 12 14 21 23 25',
 '0 23 25',
 '0',
 '0 6 21 23 25',
 '0 5 7 17 18 21 23 25',
 '3 5',
 '0 2 21 25',
 '7 9 18 20 23 25',
 '0 12 21 23 25',
 '0 4 5 18 19 21 25',
 '2 14 16 25',
 '0 4 5',
 '11 12 14 16 17 21 25',
 '0 5',
 '1 6 25',
 '0 2 3 5 7 8 20 23 25',
 '0 2 5 25',
 '0 16 17 21 23 25',
 '0 4 21',
 '0 11 12 21 25',
 '0 21 25',
 '0 5 19',
 '0 25',
 '0',
 '0 13 21 25',
 '0 18 19 21 25',
 '0 14 16 17 18 21 25',
 '0 5 25',
 '0 7 25',
 '0 13 21 25',
 '0 23 25 26',
 '0 3',
 '0 21 25',
 '0 1',
 '0 16 23 25',
 '6 23 25',
 '0 21 25',
 '18 19 25',
 '0 12 16 21 22 23 25',
 '6 7 20 21 23 25',
 '0 2',
 '0 2 23 25',
 '0 6 11 14 17 21 23 25',
 '0',
 '0 17 25',
 '0 5',
 '0 7 11 20 22 23 26',
 '0 21 23 25',
 '3 4 25',
 '0 21 25',
 '0 21 25',
 '12 21 23 25',
 '0 14 16 17 21 25',
 '0 2 6 16 21 25',
 '0 2 12 21 22 23',
 '0 5 16 21 25',
 '0 4 12 13 14 16 17 21 25',
 '0 7 21 23 25',
 '6 23 25',
 '0 16 17 18 19 21 23 

In [23]:
submit['Predicted'] = predicted

In [ ]:
# Create submit
# submit = pd.read_csv('../data/sample_submission.csv')
# predicted = []
# draw_predict = []
# # model.load_weights('../cache/InceptionV3.h5')
# for name in tqdm(submit['Id']):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
#     label_predict = np.arange(28)[score_predict>=0.2]
#     str_predict_label = ' '.join(str(l) for l in label_predict)
#     predicted.append(str_predict_label)

# submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-8.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

In [24]:
submit.to_csv('../submissions/sub10.csv', index=False)

In [25]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [26]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-10-22 22:46:14.145230


In [27]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub10.csv -m ""

Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 343 ms, sys: 185 ms, total: 528 ms
Wall time: 16.2 s


In [28]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName   date                 description  status    publicScore  privateScore  
---------  -------------------  -----------  --------  -----------  ------------  
sub10.csv  2018-10-22 17:16:40               complete  0.336        None          
sub9.csv   2018-10-21 20:04:09               complete  0.098        None          
sub9.csv   2018-10-21 19:44:17               complete  0.073        None          
sub9.csv   2018-10-21 19:37:30               complete  0.043        None          
sub9.csv   2018-10-21 19:27:51               complete  0.000        None          
sub8.csv   2018-10-20 20:08:45               complete  0.422        None          
sub7.csv   2018-10-20 17:06:09               complete  0.389        None          
sub5.csv   2018-10-19 18:27:33               complete  0.387        None          
sub4.csv   2018-10-19 14:45:15               complete  0.411        None          
sub3.csv   2018-10-19 10:19:26               complete  0.377        None          
sub2